### A/B testing

We perform A/B testing on a simple dataset to find out if changes in the landing page actually improves the conversion rate.

In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms

from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [2]:
# Import dataset
df = pd.read_csv('./ab_test.csv', index_col = 0)   

#### Exploratory data analysis - I

The idea of this section is to find out a few things about our dataset before performing the actual A/B testing.

Some preliminary findings:
1. Besides the user id index, there are 4 columns: a *time* column with a weird formatting that will be dropped; a *con_treat* column that tells whether user is in the control group or treatment group and it is either control or treatment; a *page* column that tells whether the user visited the old page or the new page; and, finally, the *converted* variable that tells whether there was actually a conversion or not, taking 1 for yes and 0 for no.
2. There are 294478 observations in the dataset.
3. Only the 11.965% of users turned out to be conversions.
4. There seems to be users, 3893 in total, that while assigned to either control or treatment group they landed on a page that was not meant for them. That is, some users got the old landing page when they should have landed on the new one since they had been assigned to the control group. This users will be deleted from the dataset. In particular, there are 1928 that were assigned to the control group and landed to the new page, and 1965 that were assigned to the treatment and landed to the old page.

In [3]:
# Some EDA
df.info()
df.shape
df.columns

df.isna().sum()

df['con_treat'].unique()
df['page'].unique()
df['converted'].unique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294478 entries, 851104 to 715931
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   time       294478 non-null  object
 1   con_treat  294478 non-null  object
 2   page       294478 non-null  object
 3   converted  294478 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 11.2+ MB


array([0, 1], dtype=int64)

In [4]:
# Mismatch between assigned group and landing page
df[(df['con_treat'] == 'control') & (df['page'] == 'new_page')].shape[0]
df[(df['con_treat'] == 'treatment') & (df['page'] == 'old_page')].shape[0]

# Delete mismatch users
df = df[(df["con_treat"] == "treatment") & (df["page"] == "new_page") |(df["con_treat"] == "control") & (df["page"] == "old_page")]

#### Exploratory data analysis - II

Some other findings:
1. There is one duplicated user ID assigned to the treatment group and it seems to have visited the new page twice, but without converting. We will not drop this observation since it could be valuable information that an user chose twice not to convert.
2. There are 145274 users assigned to the control group and therefore land on the old page, while there are 145311 users assigned to the treatment group and therefore land on the new page. There is a slight difference in size group, but since we are talking about a difference of less than 50 users in a sample of almost 300000 users it shouldn't be a problem. Statistical tests can still provide reliable results despite this minor difference.
3. In the control group 12.03% of users converted (17489 out of 145274), while in the treatment group only 11.88% of users converted (17264 out of 145311). It would seem that the new landing page performs worse than the old one, but it is not that big of a difference.

In [5]:
# Duplicated IDs
sum(df.index.duplicated())
dup_id = df.index[df.index.duplicated()][0]
df.loc[dup_id]

# Group by treatment/landing page to get proportions of conversions
pd.crosstab(index = df['con_treat'], columns = df['page'])
df.groupby('con_treat').sum()
df.groupby('con_treat').mean()

,converted
con_treat,
control,0.120386
treatment,0.118807


#### Hypothesis testing

The idea of this section is to actually find out whether the slight conversion rate difference find earlier is statistically significant or is only due to noise and it is well within the margin of error. Things being as they are right now, there seems to be only one conclussion: the new landing page is not working as expected. That is, it has either had no effect whatsoever on user behaviour or it has worsen the convertion rate performance which is definitely not the expected outcome. At this point we could just finish the experiment, there isn't anything to test really, at least not in the sense of finding out if the new landing page has a better performance than the old one. But for the sake of practice we will go on to see if this conclusion is statistically based.

For this test, we will perform a two-tailed test with a confidence level set to 95% as is it standard. We will contrast the null hypothesis of no effect (that is, conversion rates are statistically the same) against the alternative hypothesis that there is actually a difference. 

It is usually best practice to learn whether the amount of data gathered is enough and representative of the population to draw reliable conclusions. Naturally, the bigger the size of the sample the better, but sometimes it can be really expensive to gather that much data, so we need to know where we stand with the collected data. Therefore, what we will do first is perform a power analysis which is the calculation used to estimate the smallest sample size needed for an experiment, given a required significance level, statistical power, and effect size.

To perform the power analysis we need first to define some things:
1. Power of the test, that is, the probability of finding a statistical difference between groups in our test when the difference is actually there. In other words, the probability of rejecting the null hypothesis when the alternative is true. By convention, it is usually set to 0.8.
2. Alpha value which is related to the confidence interval set earlier to 5% (or 0.05).
3. Effect size relates to how big of a difference we expect to be between both groups. Since this hasn't been defined before, we will set it to 1%. When comparing two proportions we need to calculate the proportion effect size (we need this value to input to the power function).

In [6]:
# Power analysis
control_conversion_rate = df[df['con_treat'] == 'control']['converted'].mean()
power = 0.8
alpha = 0.05
effect_size = sms.proportion_effectsize(control_conversion_rate, control_conversion_rate + 0.01) 

required_size = sms.NormalIndPower().solve_power(effect_size, power = power, alpha = alpha, ratio = 1)
round(required_size)

17209

It seems we need at least a sample of size 17209 per group to find the 1% effect in either direction with a confindence of 95%. Since we have almost 150k observations for each group we are more than ok and it is safe to conduct the experiment to test whether the new landing page can increase the conversion rate by 1%.

Let's perform now the hypothesis test using the z-test to compute the p-value since the sample size is quite large.

In [7]:
# Split by control and treatment
df_control = df[df['con_treat'] == 'control']
df_treatment = df[df['con_treat'] == 'treatment']

# Get sample size
obs_control = df_control.shape[0]
obs_treatment = df_treatment.shape[0]

# Get amount of conversions in each group
conv_control = df_control['converted'].sum()
conv_treatment = df_treatment['converted'].sum()

zstat, pval = proportions_ztest(count = np.array([conv_control, conv_treatment]), nobs = np.array([obs_control, obs_treatment]))

(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(count = [conv_control, conv_treatment], nobs = [obs_control, obs_treatment], alpha = 0.05)

print(f'The p-value is {round(pval, 3)}.')
print(f'The confidence interval for the control group is [{round(lower_con, 3)}, {round(upper_con, 3)}].')
print(f'The confidence interval for the treatment group is [{round(lower_treat, 3)}, {round(upper_treat, 3)}].')

The p-value is 0.19.
The confidence interval for the control group is [0.119, 0.122].
The confidence interval for the treatment group is [0.117, 0.12].


#### Conclusions
So, the p-value is 0.189, which is above our 0.05 threshold. Therefore, there isn't enough evidence to reject the null hypothesis that both samples are statistically the same. In other words, the new landing page didn't perform significantly different from the old page. The confidence interval for the control group is [0.119, 0.122] and for the treatment group is [0.117, 0.12].

The following may be concluded:
1. There isn't enough evidence to suggest significant change between using one page design or the other. If the new page was meant to increase conversions, then it isn't working. But it could also be the case that a much more simpler and less expensive page is being used and we were interested only in knowing if such changes wouldn't at least diminish the conversion rate. Therefore, if these are good results or not depends on the objective of the page design change.
2. Before conducting any test, we knew that conversions hadn't increased, but that didn't mean that it was performing worse that the old page design, that's why we had to conduct the experiment.
3. We ought to remember that we were aiming for a 1% increase in conversion relative to the 12.03% conversion rate in the control group. From the confidence interval in the treatment group that 1% increase is not even included.

Some possible extensions:
1. Analyze performance based on time column.
2. Add more variables to group by, for example, age or city/country. Overall, it may be the case that there is no significant increase in the conversion rate, but for certain audiencies there might be such an increase. Also, we might able to detect in which audience the new page is driving the decrease in conversion rate.